In [38]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
import matplotlib.pyplot as plt


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


In [39]:
# Read in the data 

df = pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv',index_col = 'Id')

In [40]:
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [79]:
df.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuilt    

Problem Statement

Predict the sales price for each house. Using supervised ML as we want to predict a range of values.

Exploratory Data Analysis

First thing is to determine what features are most relevant. Use domain knowledge to pick out which ones you think will be good.

1. LotArea
2. YearBuilt
3. FullBath
4. Bedroom


In [81]:
X = df[['LotArea','YearBuilt','FullBath','BedroomAbvGr','GarageArea','GrLivArea','PoolArea','SaleCondition','']]
y = df['SalePrice']

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state=42)

In [10]:
model = LinearRegression()
model.fit(X_train,y_train)

LinearRegression()

In [11]:
y_pred = model.predict(X_test)

In [12]:
mse = mean_squared_error(y_test,y_pred)
r2 = r2_score(y_test,y_pred)

In [13]:
print(mse)

3408766081.0569625


In [14]:
print(r2)

0.5555904678652472


These are my scores based on a Linear Regression model and 4 input features. Not great. Could probably improve the score by adding more features but how many should we add. 

I think first we have to do some feature engineering.

1. Start off with normalizing the input features.

In [15]:
# Normalizing the features
LotAreaMean = df['LotArea'].mean()
LotAreaStd = df['LotArea'].std()
LotAreaZ_scores = (df['LotArea'] - LotAreaMean) / LotAreaStd

In [16]:
LotAreaZ_scores

Id
1      -0.207071
2      -0.091855
3       0.073455
4      -0.096864
5       0.375020
          ...   
1456   -0.260471
1457    0.266316
1458   -0.147760
1459   -0.080133
1460   -0.058092
Name: LotArea, Length: 1460, dtype: float64

Easier way to scale/normalize the data

In [66]:
scaler = StandardScaler()
X_norm = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(f"Peak to Peak range by column in Raw    X:{np.ptp(X_train,axis=0)}")
print(f"Peak to Peak range by column in Normalized    X:{np.ptp(X_norm,axis=0)}")

Peak to Peak range by column in Raw    X:LotArea         213945
YearBuilt          138
FullBath             3
BedroomAbvGr         8
GarageArea        1418
dtype: int64
Peak to Peak range by column in Normalized    X:[19.89305327  4.50063201  5.48768887  9.94393737  6.7202203 ]


In [67]:
sgdr = SGDRegressor(max_iter= 1000)
sgdr.fit(X_norm,y_train)
print(sgdr)
print(f"number of iterations completed: {sgdr.n_iter_}, number of weight updates: {sgdr.t_}")

SGDRegressor()
number of iterations completed: 19, number of weight updates: 22193.0


In [68]:
b_norm = sgdr.intercept_
w_norm = sgdr.coef_
print(f"model parameters:                   w: {w_norm}, b:{b_norm}")

model parameters:                   w: [12635.01950596 16731.07031537 20016.4605736   3345.46375069
 29893.09238037], b:[181204.37550757]


In [69]:
## Make predictions

y_pred_sgd = sgdr.predict(X_norm)
y_pred = np.dot(X_norm,w_norm) + b_norm
print(f"prediction using np.dot() and sgdr.predict match: {(y_pred == y_pred_sgd).all()}")

print(f"Prediction on training set:\n{y_pred[:6]}" )
print(f"Prediction on training set:\n{y_pred_sgd[:6]}" )
print(f"Target values \n{y_train[:6]}")

prediction using np.dot() and sgdr.predict match: True
Prediction on training set:
[124398.20718311 192178.68687638  53385.94578394 104425.12641233
 140995.25195001 170778.67431558]
Prediction on training set:
[124398.20718311 192178.68687638  53385.94578394 104425.12641233
 140995.25195001 170778.67431558]
Target values 
Id
255     145000
1067    178000
639      85000
800     175000
381     127000
304     149900
Name: SalePrice, dtype: int64


In [72]:
y_pred_sgd = sgdr.predict(X_test_scaled)
print(y_pred[:6])
print(y_train[:6])

[123438.48259275 244950.38282955 113878.30182178 169412.36545388
 276973.76882189  74498.35565641]
Id
255     145000
1067    178000
639      85000
800     175000
381     127000
304     149900
Name: SalePrice, dtype: int64


In [77]:
mse_sgd = mean_squared_error(y_test,y_pred_sgd)
rmse_test = np.sqrt(mse_sgd)
r2_sgd = r2_score(y_test,y_pred_sgd)
print(mse_sgd)
print(rmse_test)
print(r2_sgd)

3407944583.43348
58377.60344030474
0.555697568606636


Get similar MSE and R2 score for both LinearRegression and SGDRegressor. They are both for linear regression but are based on different algorithms. LinearRegression is based on OLS and SGDRegressor is based on Stochastic Gradient Descent.

To improve score:
1. Add more features
2. Create polynomial features (add regularization to prevent overfitting)
3. Use GridSearch for Hyperparameter tuning